# TBC
- output a csv per metric:
    - euc
    - dtw
    - soft-dtw
- one row for each algorithm, k, cluster
- each row has center and metric scrore to center
    - Include min, max, avg, quantile scores?

# Clustering Centers and Scores
### $Time$ $Series$ $4th$ $Test$

$Vasco$ $Mergulhão$ $-$ $April$ $2023$

### Version 1:
This script calcules the clusters centers and respective scores according to different simillarity measures:
- Euclidean
- DTW
- soft-DTW

In [104]:
from dtaidistance.dtw_barycenter import dba_loop as dba_centre
from tslearn.barycenters import softdtw_barycenter as soft_dba_centre
from tslearn.barycenters import euclidean_barycenter as euc_center

from dtaidistance.ed import distance as euc_dist

In [2]:
import pandas as pd
import polars as pl
import numpy as np
import time
import matplotlib.pyplot as plt
import Transform

---
# Script Variabels

In [11]:
dataset_name = 'Kenya_90k_Set_1_w90_pW'
solution_name = 'LSTM_AE-efficient_sweep_2_v49'

In [42]:
dataset_folder = "_".join(dataset_name.split('_')[:-2]) 
dataset_location = f'../Data_Storage_Processing/Data/{dataset_folder}/{dataset_name}.csv'

sol_location = f'../ModelResults/Clustering/Solutions/{dataset_name}/{solution_name}.csv'

window_cols, window_len = Transform.retrive_window_col_names(dataset_name)

---
# Data Imports

In [6]:
df_data = pl.read_csv(dataset_location)

---
# Euclidean
https://dtaidistance.readthedocs.io/en/latest/usage/ed.html  
https://tslearn.readthedocs.io/en/stable/gen_modules/barycenters/tslearn.barycenters.euclidean_barycenter.html#tslearn.barycenters.euclidean_barycenter

In [110]:
sol_cols = pl.scan_csv(sol_location).columns[5:]
centroids = []
algorithms = []
k = []
clusters = []
avg_scores = []
for s, sol in enumerate(sol_cols):
    # Getting range of clusters for sol
    c_min = pl.scan_csv(sol_location).select(pl.min(f'{sol}')).collect().item()
    c_max = pl.scan_csv(sol_location).select(pl.max(f'{sol}')).collect().item()

    # Calculating Centroid/Barycenter per cluster per sol.
    for c in range(c_min, c_max + 1):
        # Getting relevant windows (cluster points)
        df_sol_ids = pl.scan_csv(sol_location).filter(pl.col(f"{sol}") == c).select(["short_ID", "window_ID"]).collect()
        df_sol = df_sol_ids.join(df_data, on = ['short_ID', 'window_ID'], how='left').to_pandas()
        sol_data = df_sol[window_cols][:50].values
        
#         print(f'Processing {sol} ({s+1}/{len(sol_cols)}) cluster {c} ({c}/{c_max})' , end='\r')
#         # Calculating cluester centre
#         centroid = euc_center(sol_data).ravel()
#         # Saving cluster centre
#         centroids.append(centroid)
        
#         # Saving Algorithm and k values
#         algorithms.append(sol.split('_')[0])
#         k.append(sol.split('_')[-1])
#         clusters.append(c)
        
#         Euclidean_centres = pd.DataFrame(centroids, columns=window_cols)
#         Euclidean_centres.insert(0, "cluster", clusters)
#         Euclidean_centres.insert(0, "k", k)
#         Euclidean_centres.insert(0, "Algorithm", algorithms)
        
        # Calculating Average Score for respective metric
        scores = []
        for window in sol_data:
            score = euc_dist(centroid, window)
            scores.append(score)
        avg_scores.append(np.array(scores).mean())
print(avg_scores)
            


[14.24841198265961, 85.38149682454625, 82.301506559046, 85.38149682454625, 14.24841198265961, 14.31441276923223, 82.301506559046, 21.331010632375786, 85.38149682454625, 21.331010632375786, 85.38149682454625, 82.23952500959403, 14.31441276923223, 82.17687051262544, 21.331010632375786, 85.38149682454625, 82.36303654169717, 14.31441276923223, 82.16903031148507, 82.1266165274308, 14.603320693492424, 85.38149682454625, 82.1266165274308, 82.16903031148507, 82.36303654169717, 23.25141107123977, 17.63270169166352, 18.283544359830827, 85.38149682454625, 82.12695264573294, 82.36303654169717, 16.5206861285421, 14.788320373844858, 23.48843447405791, 82.15885614828416, 11.906789872891558, 85.38149682454625, 18.098755394592917, 82.24228229640683, 81.77894539142373, 23.496927439150646, 15.087865178353963, 82.43134917522052, 82.23641032110945, 14.922497681560547, 85.38149682454625, 17.116129176830697, 82.24223404285489, 11.668319765821803, 82.19784112329664, 23.57746437687086, 18.58000022810995, 82.44

In [81]:
Euclidean_centres

,Algorithm,k,cluster,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,kMeans,k2,1,0.748996,0.677126,0.605850,0.625368,0.609416,0.576102,0.583896,...,0.586845,0.466476,0.546499,0.591705,0.532292,0.560615,0.554449,0.549619,0.551327,0.568321
1,kMeans,k2,2,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000
2,kMeans,k3,1,-4.486170,-4.521182,-4.576265,-4.562245,-4.774801,-5.148132,-5.697935,...,-6.851132,-6.820000,-6.640000,-6.280000,-6.107416,-5.967560,-6.100000,-5.920000,-5.762592,-5.622592
3,kMeans,k3,2,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000
4,kMeans,k3,3,0.748996,0.677126,0.605850,0.625368,0.609416,0.576102,0.583896,...,0.586845,0.466476,0.546499,0.591705,0.532292,0.560615,0.554449,0.549619,0.551327,0.568321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,KUMAP,k25,21,1.272262,1.368280,1.317439,1.288939,1.368183,1.308562,1.315617,...,1.034565,1.096173,1.071781,1.026973,1.084051,1.011629,0.966491,0.978620,0.951705,1.028711
644,KUMAP,k25,22,-6.533870,-7.000000,-6.460000,-6.289836,-6.129836,-6.304028,-6.344028,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-6.640000
645,KUMAP,k25,23,-6.526373,-6.846669,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,2.000000,1.923390,1.771619
646,KUMAP,k25,24,-3.009853,-4.468448,-6.280000,-6.280000,-6.280000,-6.280000,-6.304865,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-6.820000,-6.820000,-6.824347,-6.844347,-7.000000


In [105]:
# distance = ed.distance(s1, s2)
scores = []
for index, row in Euclidean_centres.iterrows():
    centre = row[window_cols].values
    sol = "_".join([row['Algorithm'], row['k']])
    c = row['cluster']
    df_sol_ids = pl.scan_csv(sol_location).filter(pl.col(f"{sol}") == c).select(["short_ID", "window_ID"]).collect()
    df_sol = df_sol_ids.join(df_data, on = ['short_ID', 'window_ID'], how='left').to_pandas()
    
    print(index)
    score  = df_sol[window_cols].apply(lambda x: euc_dist(x, centre), axis=1).values
    # scores.append(centre, )
    print(score.mean())

0
8.390659990138158
1
1.664683193898194
2
20.674370437523365
3
0.25045373934491566
4


KeyboardInterrupt: 

In [102]:
score.shape

(8191,)